# Importing all necessary packages

In [24]:
import sys
sys.path.append('../../src/')

In [25]:
import numpy as np
import pandas as pd
import scipy.stats
from fitter import Fitter
from collections import defaultdict, OrderedDict
import pylogit as pl
from functools import reduce
import seaborn as sns
import random
from scipy import sparse
import copy
from tqdm import tqdm, tqdm_notebook
import viz

# Distribution Fitting Class Definition - **Could be removed** - For Documentation Purposes

In [ ]:
# class FitDistribution(object):
#     """Fit and simulate data to known distributions.

#     Input:
#     ------
#     - data: array-like or dataframe.
#     - dists: list.
#         This parameter contains a list of distributions to be explored.
#         When None, every available distribution on scipy is explored.
#     - bins: int.
#         Numbers of bins to be used for the cumulative histogram. This has
#         an impact on the quality of the fit.
#     - timeout: int.
#         Maximum time for a given distribution. If timeout is reached,
#         the distribution is skipped.
#         """
#     def __init__(self, data, dists=None, timeout=30, verbose=False, bins=100):
#         self.data = data
#         # self.var_types = var_types
#         self.dists = dists
#         self.timeout = timeout
#         self.verbose = verbose
#         self.bins = bins
#         self.ArrayDistDict = defaultdict()
#         self.params_dict = defaultdict(dict)

#     def FindArrayDist(self, cat_var):
#         """Function to extract the best distribution for a specified array.
#         Uses the fit method from the Fitter module in the fitter library
#         Inputs:
#         -------
#         - cat_var: boolean
#             Boolean to signify whether the variable to be simulated
#             is discrete/categorical or continuous.

#         Outputs:
#         -------
#         By default, the function returns a dictionary with best distribution
#         name and parameters associated with it. If a number of distributions
#         was specified, the function returns a pandas DataFrame with
#         the N best distributions, along with a plot showing all of them."""
#         self.ArrayDistDict = dict()
#         if cat_var is True:
#             self.ArrayDistDict['distribution'] = 'categorical'
#             np_array_range = np.arange(self.data.max()+1)
#             array_bincount = np.bincount(self.data)
#             probs = array_bincount / len(self.data)

#             self.ArrayDistDict['parameters'] = [np_array_range,
#                                                 probs]
#         else:
#             fitter_object = Fitter(data=self.data,
#                                    distributions=self.dists,
#                                    timeout=self.timeout)
#             fitter_object.fit()
#             BestDict = fitter_object.get_best()
#             self.ArrayDistDict['distribution'] = list(BestDict.items())[0][0]
#             self.ArrayDistDict['parameters'] = list(BestDict.items())[0][1]
#         return self.ArrayDistDict

#     def SimArray(self, size=100):
#         """Function to simulate data for an array based on the best fitted
#         distribution.
#         Input:
#         -----
#         - size : int
#                 size of the array to be simulated.
#         Outputs:
#         -------
#         Simulated array based on the best fit distribution."""
#         if self.ArrayDistDict['distribution'] == 'categorical':
#             value = self.ArrayDistDict['parameters'][0]
#             freq = self.ArrayDistDict['parameters'][1]
#             Sim_Array = np.random.choice(a=value,
#                                          p=freq,
#                                          size=size)
#         else:
#             dist = getattr(scipy.stats, self.ArrayDistDict['distribution'])
#             Sim_Array = dist.rvs(*self.ArrayDistDict['parameters'], size=size)
#         return Sim_Array

#     def FindDfDist(self, var_types):
#         """Function to extract the best distribution from a specified
#         dataframe. Uses the function find_dist, which in turn uses the
#         fit method from the Fitter module in the fitter library
#         Inputs:
#         -------
#         - var_types: dictionary
#             Dictionary with keys as column names for dataset variables,
#             the value of each key is a string showing whether the
#             variable is discrete/cat or continuous.

#         Outputs:
#         -------
#         *FOR NOW*, the function returns a dictionary showing the best
#         distribution name for each array in the dataframe and parameters
#         associated with it.
#         """

#         for column in list(self.data.columns):

#             if var_types[column] == 'categorical':
#                 if len(self.data[column].unique()) == 1:
#                     self.params_dict[column]['distribution'] = 'constant'
#                     self.params_dict[column]['parameters'] = \
#                         self.data[column].unique()
#                 else:
#                     self.params_dict[column]['distribution'] = 'categorical'
#                     np_array_range = np.arange(self.data[column].max()+1)
#                     array_bincount = np.bincount(self.data[column])
#                     probs = array_bincount / len(self.data[column])
#                     self.params_dict[column]['parameters'] = [np_array_range,
#                                                               probs]
#             else:
#                 if len(self.data[column].unique()) == 1:
#                     self.params_dict[column]['distribution'] = 'constant'
#                     self.params_dict[column]['parameters'] = \
#                         self.data[column].unique()
#                 else:
#                     fitter_object = Fitter(data=self.data[column],
#                                            distributions=self.dists,
#                                            timeout=self.timeout)
#                     fitter_object.fit()
#                     BestDict = fitter_object.get_best()
#                     self.params_dict[column]['distribution'] = \
#                         list(BestDict.items())[0][0]
#                     self.params_dict[column]['parameters'] = \
#                         list(BestDict.items())[0][1]
#         return self.params_dict

#     def SimDf(self, size=1000):
#         """Funtion to simulate data of size N based on specified
#         distribution/parameters found by the fitter package.
#         Inputs:
#         -------
#         data: dataframe from which columns are to be taken
#         dist_params: the distribution parameters from find_dist_df
#         Outputs:
#         -------
#         DataFrame object with simulated data based on specified distributions
#         """
#         Sim_Df = pd.DataFrame(columns=list(self.params_dict.keys()))
#         Sim_Df = Sim_Df.fillna(0)
#         for column in list(self.params_dict.keys()):
#             if self.params_dict[column]['distribution'] == 'categorical':
#                 value = self.params_dict[column]['parameters'][0]
#                 freq = self.params_dict[column]['parameters'][1]
#                 data_sim = np.random.choice(a=value,
#                                             p=freq,
#                                             size=size)
#                 Sim_Df[column] = data_sim
#             elif self.params_dict[column]['distribution'] == 'constant':
#                 data_sim = self.params_dict[column]['parameters'][0]
#                 Sim_Df[column] = data_sim
#             else:
#                 dist = getattr(scipy.stats,
#                                self.params_dict[column]['distribution'])
#                 data_sim = dist.rvs(*self.params_dict[column]['parameters'],
#                                     size=size)
#                 Sim_Df[column] = data_sim
#         return Sim_Df

# Functions to calculate probabilities for each alternative - **(Replaced by functions from TB)** - For Documentation Purposes

In [ ]:
# def add_intercept_to_df(df_long, specification_dict):
#     """Function to add intercept to long format DataFrame
#     Parameters
#     ----------
#     df_long: DataFrame
#         Long format Pandas DataFrame to which to add 
#         intercept column.
#     specification_dict: dict
#         Specification Dictionary for the model
    
#     Returns
#     -------
#     In-place Pandas DataFrame with additional intercept column.
        
#     """
#     if ("intercept" in specification_dict
#             and "intercept" not in df_long.columns):
#         df_long["intercept"] = 1
#     return None


# def create_design_matrix(df_long, specification_dict,
#                          names_dict, alternative_id_col):

#     add_intercept_to_df(df_long, specification_dict)

#     columns = []
#     for col in specification_dict:
#         for group in specification_dict[col]:
#             if type(group) == list:
#                 columns.append(df_long[alternative_id_col].isin(group)
#                                * df_long[col])
#             else:
#                 columns.append((df_long[alternative_id_col] == group)
#                                * df_long[col])

#     design_matrix = np.stack(columns, axis=1)

#     var_names = []
#     for variable in names_dict:
#         for name in names_dict[variable]:
#             var_names.append(name)

#     return design_matrix, var_names


# def calculate_utilities(betas, design_matrix):

#     limit_max = 700
#     limit_min = -700

#     utility = design_matrix.dot(betas)
#     utility[utility > limit_max] = limit_max
#     utility[utility < limit_min] = limit_min

#     utilities = np.exp(utility)

#     return utilities


# def create_mapping_matrix(df_long, observation_id_col):
#     row_to_col_matrix = pd.get_dummies(df_long[observation_id_col]).values
# #     row_to_col_matrix = (df_long[observation_id_col].values[:,None] ==
# #                          np.sort(df_long[observation_id_col].unique())[None,:]).astype(int)
#     sparse_row_to_col_matrix = sparse.csr_matrix(row_to_col_matrix)

#     mapping_matrix = sparse_row_to_col_matrix.dot(sparse_row_to_col_matrix.T)

#     return mapping_matrix


# def calculate_probabilities(betas, design_matrix, mapping_matrix):

#     utilities = calculate_utilities(betas, design_matrix)
#     denominator = mapping_matrix.dot(utilities)
#     probabilities = utilities/denominator
#     probabilities[probabilities == 0] = 1e-300

#     return probabilities

# Function to simulate choices based on long data format - For Documentation Purposes

In [ ]:


# def SimulateChoices(long_data, alt_id_col,
#                     obs_id_col, number_alts,
#                     spec_dic, names_dic, init_betas):
#     """
#     Function to simulate choices from a long data
#     format dataset.
    
#     Parameters
#     ----------
#     long_data : DataFrame
#         The DataFrame to be used, in long format.
#     alt_id_col: string
#         Name of the column containing the alternative
#         id numbers in the long format dataset.
#     obs_id_col: string
#         Name of the column containing the observation
#         id numbers in the long format dataset.
#     number_alts: int
#         Number of alternatives in the long format
#         dataset.
#     spec_dic: dictionary
#         Dictionary of the model specification.
#     names_dic: dictionary
#         Dictionary of the alternative names.
#     init_betas: list
#         List of the initial betas for the model
#         from which the choices should be simulated.
    
#     Returns
#     -------
#     DataFrame object with the simulated choices column
#     added as 'sim_choice'
#     """
#     # Declare the simulated choice column name
#     sim_choice_col = 'sim_choice'

#     # Make a copy of the data
#     data = copy.deepcopy(long_data)
#     # Functions to generate the design matrix, mapping matrix,
#     # and calculate the probabilities for each alternative
#     design_matrix, names = create_design_matrix(df_long=data,
#                                                 specification_dict=spec_dic,
#                                                 names_dict=names_dic,
#                                                 alternative_id_col=alt_id_col)
#     mapping_matrix = create_mapping_matrix(df_long=data,
#                                            observation_id_col=obs_id_col)
#     probabilities = calculate_probabilities(betas=initial_betas,
#                                             design_matrix=design_matrix,
#                                             mapping_matrix=mapping_matrix)
#     # Assign calculated probabilities to new dataframe column
#     data['probabilities'] = probabilities
#     # Initialize cumulative sum and simulated choice columns
#     data['cum_sum'] = 0
#     data['sim_choice'] = 0

#     # Loop around the observations and compute probabilities' cumulative
#     # sums for each alternative
#     for observation in data['observation_id'].unique():
#         probs_sum = data[long_data.observation_id == observation]['probabilities'].cumsum()
#         data.loc[data['observation_id'] == observation, 'cum_sum'] = probs_sum

#     # Generate list for observation ids to be used in simulating choices    
#     observation_id_list = list(data.observation_id.unique())
#     # Generate a "random utility" array of the same size as the number
#     # of observations in the dataset
#     u_random = np.random.uniform(size=len(data['observation_id'].unique()))

#     # Loop around the generate utilities and observations in the dataset
#     # to assign a choice to each
#     for u, obs in zip(u_random, observation_id_list):
#         # select data for observation number "obs"
#         data_sample = data[data['observation_id'] == obs]
#         # generate list of available modes for each observation
#         sorted_list = sorted(list(data_sample['mode_id'].unique()))
#         # initialize a dictionary from the available modes for 
#         # each observation
#         choices = dict.fromkeys(sorted_list, 0)
#         # Loop round the modes for each observation and assign 
#         # choice (0 vs. 1)
#         for alt in sorted_list:
#             choices[alt] = np.where(u <= data_sample[data_sample['mode_id']
#                                                      == alt]
#                                     ['cum_sum'], 1, 0).item()
#             # Once a choice is assigned, break out of loop
#             if choices[alt] == 1:
#                 break
#         # Map the choices for the observation to the long format dataframe       
#         data.loc[data.observation_id == obs, sim_choice_col] = \
#             data['mode_id'].map(choices)
#     return data

# Simulate Data - Based on Wide Data - For Documentation Purposes

In [ ]:
# # Convert Data from Long to Wide before simulation

# # Create variable list for subset of long data,
# # I will add the remaining values from the long format dataset
# alt_id_col = "mode_id"

# obs_id_col = "observation_id"

# choice_col = "choice"

# # Store of columns relevant to the data to be simulated
# model_variables = list(mnl_model.specification.keys())
# model_variables.remove('intercept')
# model_variables.extend([alt_id_col, obs_id_col,
#                         choice_col, 'num_cars', 'num_licensed_drivers'])
# print('The model variables of interest are:')
# model_variables

In [ ]:
# # Create a copy of the data subset
# subset_bike_data = bike_data_long[model_variables].copy()

In [ ]:
# # Create the needed variables for the conversion
# ind_spec_vars = ['num_kids', 'household_size',
#                  'cars_per_licensed_drivers', 'cross_bay',
#                  'num_cars', 'num_licensed_drivers']

# alt_spec_vars = ['total_travel_time', 'total_travel_distance']

# subset_spec_vars = {'total_travel_cost': [1, 2, 3],
#                     'cost_per_distance': [1, 2, 3]}

# alternative_name_dict = {1: 'drive_alone',
#                          2: 'shared_2',
#                          3: 'shared_3p',
#                          4: 'wtw',
#                          5: 'dtw',
#                          6: 'wtd',
#                          7: 'walk',
#                          8: 'bike'}

In [ ]:
# # Convert data from long to wide, I assigned a null value of 0
# # because it will make it easier to simulate data when we have
# # Unavailable variables
# bike_data_wide = pl.convert_long_to_wide(long_data=subset_bike_data,
#                                          ind_vars=ind_spec_vars,
#                                          alt_specific_vars=alt_spec_vars,
#                                          subset_specific_vars=subset_spec_vars,
#                                          obs_id_col=obs_id_col,
#                                          alt_id_col=alt_id_col,
#                                          choice_col=choice_col,
#                                          alt_name_dict=alternative_name_dict)
# bike_data_wide.head()

In [28]:
# Here, we need to decide how we will simulate data when we have unavailable values. TBD.

In [ ]:
# # Define the list of variables of interest from data_wide
# columns_wide = ['num_kids', 'household_size', 'num_cars',
#                 'num_licensed_drivers', 'cross_bay',
#                 'total_travel_time_drive_alone', 'total_travel_time_shared_2',
#                 'total_travel_time_shared_3p', 'total_travel_time_wtw',
#                 'total_travel_time_dtw', 'total_travel_time_wtd',
#                 'total_travel_time_walk', 'total_travel_time_bike',
#                 'total_travel_distance_drive_alone',
#                 'total_travel_distance_shared_2',
#                 'total_travel_distance_shared_3p',
#                 'total_travel_distance_wtw',
#                 'total_travel_distance_dtw', 'total_travel_distance_wtd',
#                 'total_travel_distance_walk', 'total_travel_distance_bike',
#                 'total_travel_cost_drive_alone', 'total_travel_cost_shared_2',
#                 'total_travel_cost_shared_3p']

# # Restrict data to desired columns
# bike_data_wide = bike_data_wide[columns_wide]

In [ ]:
# # Determine the distributions to be used
# distributions = ['normal', 'alpha', 'beta', 'gamma', 'expon', 'gumbel']

# # Initial the FitDistribution object
# bike_data_fitter = FitDistribution(data=bike_data_wide, dists=distributions)

In [ ]:
# # Define the nature of each variables whether
# # discrete/categorical or continuous
# variable_type = {'num_kids': 'categorical',
#                  'household_size': 'categorical',
#                  'num_cars': 'discrete',
#                  'num_licensed_drivers': 'categorical',
#                  'cross_bay': 'categorical',
#                  'total_travel_time_drive_alone': 'continuous',
#                  'total_travel_time_shared_2': 'continuous',
#                  'total_travel_time_shared_3p': 'continuous',
#                  'total_travel_time_wtw': 'continuous',
#                  'total_travel_time_dtw': 'continuous',
#                  'total_travel_time_wtd': 'continuous',
#                  'total_travel_time_walk': 'continuous',
#                  'total_travel_time_bike': 'continuous',
#                  'total_travel_distance_drive_alone': 'continuous',
#                  'total_travel_distance_shared_2': 'continuous',
#                  'total_travel_distance_shared_3p': 'continuous',
#                  'total_travel_distance_wtw': 'continuous',
#                  'total_travel_distance_dtw': 'continuous',
#                  'total_travel_distance_wtd': 'continuous',
#                  'total_travel_distance_walk': 'continuous',
#                  'total_travel_distance_bike': 'continuous',
#                  'total_travel_cost_drive_alone': 'continuous',
#                  'total_travel_cost_shared_2': 'continuous',
#                  'total_travel_cost_shared_3p': 'continuous'}

In [ ]:
# # Simulate dataframe based on the estimated distributions
# sim_bike_data = bike_data_fitter.SimDf(size=5000)

## Find Distribution Based on Long Data

In [ ]:


def FindLongDataDist(data_long,
                     alt_id_col,
                     obs_id_col,
                     alt_spec,
                     alt_name_dic,
                     ind_spec,
                     trip_spec,
                     var_types,
                     cont_dists=None):
    
    """
    Function to find the distribution of specific variables
    from a long format dataset.
    
    Parameters
    ----------
    data_long: Pandas DataFrame
        Dataset in long format from which variable
        distribution is to be found.
        
    alt_id_col: string
        Name of the column with alternative ids.
        
    obs_id_col: string
        Name of the column with observation ids.
        
    alt_spec: list
        List containing strings of the names of
        alternative specific variables.
        
    alt_name_dic: dictionary
        Dictionary with keys as the ordered number
        of alternatives, and the value for each key
        is a string representing the name of the 
        alternative.
        
    ind_spec: list
        List containing strings of the names of 
        individual specific variables.
        
    trip_spec: list
        List containing string of the names of 
        trip specific variables.
        
    var_types: dictionary
        Dictionary with keys as strings of names of variables
        from long format dataset, and values for each key are
        the type of variables (e.g.: 'categorical vs. continuous').
        
    cont_dists: list
        List of continuous RVs distribution names from scipy.
        
    Returns
    -------
    a nested dictionary with keys as variable names and values
    as dictionaries containing both the distribution name and
    its parameters.
    """

    # Initialize the output parameters dictionary
    params_dict = defaultdict(dict)

    # Loop around individual specific variables
    for ind in ind_spec:
        # generate array of values for individual specific variable
        ind_var = pd.Series([(data_long.loc[data_long[obs_id_col] == x][ind].unique()[0]) for x in data_long[obs_id_col].unique()])
        # Get distribution if variable is categorical
        if var_types[ind] == 'categorical':
            # If only one category
            if len(ind_var.unique()) == 1:
                params_dict[ind]['distribution'] = 'constant'
                params_dict[ind]['parameters'] = ind_var.unique()
            # If more than one category
            else:
                params_dict[ind]['distribution'] = 'categorical'
                # Count frequency of values and store it as paramater of distribution
                np_array_range = np.arange(ind_var.max()+1)
                array_bincount = np.bincount(ind_var)
                probs = array_bincount / len(ind_var)
                params_dict[ind]['parameters'] = [np_array_range,
                                                  probs]
        else:
            # If not categorical but just one unique value
            if len(ind_var.unique()) == 1:
                params_dict[ind]['distribution'] = 'constant'
                params_dict[ind]['parameters'] = ind_var.unique()
            # If not categorical but not one unique value
            else:
                # Use the Fitter library to fit distributions
                # to the data
                fitter_object = Fitter(data=ind_var,
                                       distributions=cont_dists,
                                       timeout=30)
                fitter_object.fit()
                # Get the best distribution and store in dictionary
                BestDict = fitter_object.get_best()
                params_dict[ind]['distribution'] = list(BestDict.items())[0][0]
                params_dict[ind]['parameters'] = list(BestDict.items())[0][1]

    # Code for Alternative Specific Variables
    # Loop around the different available alternatives
    for alt in data_long[alt_id_col].unique():
        # Store data for specific alternative (mode)
        mode_data = data_long.loc[data_long['mode_id'] == alt]
        # Loop around the alternative specific variables in the input dictionary
        for var in alt_spec:
            # If data is categorical
            if var_types[var] == 'categorical':
                # If only one category
                if len(mode_data[var].unique()) == 1:
                    # Add name of alternative to variable and store distriburion & parameters
                    params_dict[var+'_'+alt_name_dic[alt]]['distribution'] = 'constant'
                    params_dict[var+'_'+alt_name_dic[alt]]['parameters'] = mode_data[var].unique()
                else:
                    # If more than one category, compute the frequency of values
                    # and store as parameters
                    # Add name of alternative to variable and store distriburion & parameters
                    params_dict[var+'_'+alt_name_dic[alt]]['distribution'] = 'categorical'
                    np_array_range = np.arange(mode_data[var].max()+1)
                    array_bincount = np.bincount(mode_data[var])
                    probs = array_bincount / len(mode_data[var])
                    params_dict[var+'_'+alt_name_dic[alt]]['parameters'] = [np_array_range,
                                                                            probs]
            else:
                # If data is not categorical but has one unique value
                if len(mode_data[var].unique()) == 1:
                    # Add name of alternative to variable and store distriburion & parameters
                    params_dict[var+'_'+alt_name_dic[alt]]['distribution'] = 'constant'
                    params_dict[var+'_'+alt_name_dic[alt]]['parameters'] = mode_data[var].unique()
                # If data is not categorical but has more than one unique value
                else:
                    # Use the Fitter library to fit distributions
                    # to the data
                    fitter_object = Fitter(data=mode_data[var],
                                           distributions=cont_dists,
                                           timeout=30)
                    fitter_object.fit()
                    # Get the best distribution and store in dictionary
                    BestDict = fitter_object.get_best()
                    # Add name of alternative to variable and store distriburion & parameters
                    params_dict[var+'_'+alt_name_dic[alt]]['distribution'] = list(BestDict.items())[0][0]
                    params_dict[var+'_'+alt_name_dic[alt]]['parameters'] = list(BestDict.items())[0][1]

    # Trip Specific Variable (maybe combine with individual specific variables)
    # Loop around trip (observation) specific variables
    for var in trip_spec:
        # generate array of values for trip specific variable
        trip_var = pd.Series([(data_long.loc[data_long[obs_id_col] == x][var].unique()[0]) for x in data_long[obs_id_col].unique()])
        # Get distribution if variable is categorical
        if var_types[var] == 'categorical':
            if len(trip_var.unique()) == 1:
            # If only one category
                params_dict[var]['distribution'] = 'constant'
                params_dict[var]['parameters'] = trip_var.unique()
            else:
            # If more than one category
                params_dict[var]['distribution'] = 'categorical'
            # Count frequency of values and store it as paramater of distribution
                np_array_range = np.arange(trip_var.max()+1)
                array_bincount = np.bincount(trip_var)
                probs = array_bincount / len(trip_var)
                params_dict[var]['parameters'] = [np_array_range,
                                                  probs]
        else:
            # If not categorical but just one unique value
            if len(trip_var.unique()) == 1:
                params_dict[var]['distribution'] = 'constant'
                params_dict[var]['parameters'] = trip_var.unique()
            # If not categorical but just one unique value
            else:
                # Use the Fitter library to fit distributions
                # to the data
                fitter_object = Fitter(data=trip_var,
                                       distributions=cont_dists,
                                       timeout=30)
                fitter_object.fit()
                # Get the best distribution and store in dictionary
                BestDict = fitter_object.get_best()
                params_dict[var]['distribution'] = list(BestDict.items())[0][0]
                params_dict[var]['parameters'] = list(BestDict.items())[0][1]

    return params_dict


def SimDf(params_dict, size=1000):
    """
    Funtion to simulate data of size N based on specified
    distribution/parameters found by the fitter package.
    
    Paremeters
    ----------
    dist_params: dictionary
        The variable distribution dictionary resulting from
        `FindLongDataDist`.
        
    size: int
        Size of the desired simulated dataset, default value
        is 1000 observations.
    
    Returns
    -------
    DataFrame object with simulated data based on specified distributions
    
    """
    # Create Empty DataFrame with keys from params_dict
    Sim_Df = pd.DataFrame(columns=list(params_dict.keys()))
    Sim_Df = Sim_Df.fillna(0)
    
    # Loop around each of the variables in params_dict
    # and simulate data for them
    for column in list(params_dict.keys()):
        if params_dict[column]['distribution'] == 'categorical':
            data_sim = np.random.choice(a=params_dict[column]['parameters'][0],
                                        p=params_dict[column]['parameters'][1],
                                        size=size)
            Sim_Df[column] = data_sim
        elif params_dict[column]['distribution'] == 'constant':
            data_sim = params_dict[column]['parameters'][0]
            Sim_Df[column] = data_sim
        else:
            dist = getattr(scipy.stats, params_dict[column]['distribution'])
            data_sim = dist.rvs(*params_dict[column]['parameters'], size=size)
            Sim_Df[column] = data_sim
    return Sim_Df


def SimulateAvailability(data_long, sim_data, obs_id_col, alt_name_dict):
    
    """
    Function to simulate alternative availability based on a long format
    dataset and join the availability data to the simulated dataset
    resulting from SimDf.
    
    Parameters
    ----------
    data_long: Pandas DataFrame
        Long format dataframe used for simulating
        alternative availability.
    
    sim_data: Pandas DataFrame
        Wide format dataframe resulting from SimDf
    
    obs_id_col: string
        Name of the column in data_long with 
        observation ids.
    
    alt_name_dic: dictionary
        Dictionary with keys as the ordered number
        of alternatives, and the value for each key
        is a string representing the name of the 
        alternative.
        
    Returns
    -------
    Wide format Pandas DataFrame with additional availability
    columns for each of the alternatives.
    
    """

    # Create empty Series to hold number of available alternatives
    series = pd.Series([])
    
    # Loop around each observation to record the
    # available number of alternatives for each observation
    for i, obs in zip(np.arange(len(data_long[obs_id_col].unique())), data_long[obs_id_col].unique()):
        series[i] = data_long[data_long[obs_id_col] == obs].shape[0]

    # Simulate number of available alternatives for
    # each observation in sim_data
    av_size = sim_data.shape[0]
    alts_sim = np.random.choice(a=np.arange(series.max()+1),
                                p=np.bincount(series)/len(series),
                                size=av_size)

    # simulate the availability matrix based on number 
    # of available alternatives
    N = len(alt_name_dict)
    av_sim = [np.array([1] * K + [0]*(N-K)) for K in alts_sim]

    # Shuffle the available alternatives for each observation
    # because av_sim will always start with 1s
    for x in av_sim:
        np.random.shuffle(x)

    # Shuffle the availability across different observations
    np.random.shuffle(av_sim)
    
    # Create columns for the availability matrix
    AV_columns = [alt_name_dict[i]+'_AV' for i in alt_name_dict.keys()]
    
    # Create alternative availability matrix with AV_columns
    AV_Df = pd.DataFrame(av_sim, columns=AV_columns)
    
    # Create an random choice column based on available
    # alternatives for each observation - This column will
    # be needed when converting to long data
    fake_choice = [random.choice(np.nonzero(a == 1)[0]) + 1 for a in np.array(AV_Df)]
    fake_choice_df = pd.DataFrame(fake_choice, columns=['sim_choice'])
    
    # Concatenate the simulated data with availability data and fake choice data
    # and return Sim_DF_AV
    Sim_DF_AV = pd.concat([sim_data, AV_Df, fake_choice_df], axis=1, sort=False)
    return Sim_DF_AV

# Example using bike data 

## Data Ingestion and Exploration 

In [26]:
# Create a variable for the path to the long format data for
# the multinomial choice model
PATH = '../../data/raw/spring_2016_all_bay_area_long_format_plus_cross_bay_col.csv'

In [27]:
# Reading data from the specified PATH
bike_data_long = pd.read_csv(PATH)

# If in previous work we accidentally saved the index with the dataframe
# remove the old index from the data
if "Unnamed: 0" in bike_data_long.columns:
    del bike_data_long["Unnamed: 0"]

print("The columns of bike_data are:")
bike_data_long.columns

The columns of bike_data are:


Index(['household_id', 'person_id', 'tour_id', 'observation_id', 'mode_id',
       'choice', 'tour_origin_taz', 'primary_dest_taz', 'total_travel_time',
       'total_travel_cost', 'total_travel_distance', 'age', 'household_size',
       'household_income', 'household_income_values', 'transit_subsidy',
       'transit_subsidy_amount', 'num_cars', 'num_licensed_drivers',
       'cross_bay', 'oakland_and_berkeley', 'survey_id', 'gender',
       'non_relative_flag', 'num_pre_school', 'num_school_aged', 'married',
       'parent', 'income_category_1', 'income_category_2', 'income_category_3',
       'income_category_4', 'income_category_5', 'income_category_6',
       'income_category_7', 'income_category_8', 'income_category_9',
       'income_category_10', 'income_unknown', 'ln_drive_cost',
       'ln_drive_cost_sq', 'total_travel_time_10x', 'total_travel_time_tenth',
       'high_income', 'medium_income', 'low_income', 'high_income_cost',
       'medium_income_cost', 'low_income_cost', 

In [ ]:
# Look at the mode shares in the data set
alt_id_to_mode_name = {1: "Drive Alone",
                       2: "Shared Ride 2",
                       3: "Shared Ride 3+",
                       4: "Walk-Transit-Walk",
                       5: "Drive-Transit-Walk",
                       6: "Walk-Transit-Drive",
                       7: "Walk",
                       8: "Bike"}

mode_counts = bike_data_long.loc[bike_data_long.choice == 1,
                                 "mode_id"].value_counts().loc[range(1, 9)]

mode_shares = mode_counts / bike_data_long.observation_id.max()
mode_shares.index = [alt_id_to_mode_name[x] for x in mode_shares.index.values]
mode_shares.name = "Mode Shares"
mode_shares

## MNL Model Specification 

In [ ]:
# Create my specification and variable names for the basic MNL model
# NOTE: - Keys should be variables within the long format dataframe.
#         The sole exception to this is the "intercept" key.
#       - For the specification dictionary, the values should be lists
#         or lists of lists. Within a list, or within the inner-most
#         list should be the alternative ID's of the alternative whose
#         utility specification the explanatory variable is entering.

mnl_specification = OrderedDict()
mnl_names = OrderedDict()

mnl_specification["intercept"] = [2, 3, 4, 5, 6, 7, 8]
mnl_names["intercept"] = ['ASC Shared Ride: 2',
                          'ASC Shared Ride: 3+',
                          'ASC Walk-Transit-Walk',
                          'ASC Drive-Transit-Walk',
                          'ASC Walk-Transit-Drive',
                          'ASC Walk',
                          'ASC Bike']

mnl_specification["total_travel_time"] = [[1, 2, 3], [4, 5, 6]]
mnl_names["total_travel_time"] = ['Travel Time, units:min (All Auto Modes)',
                                  'Travel Time, units:min (All Transit Modes)']

mnl_specification["total_travel_cost"] = [[4, 5, 6]]
mnl_names["total_travel_cost"] = ['Travel Cost, units:$ (All Transit Modes)']

mnl_specification["cost_per_distance"] = [1, 2, 3]
mnl_names["cost_per_distance"] = ["Travel Cost per Distance, units:$/mi (Drive Alone)",
                                  "Travel Cost per Distance, units:$/mi (SharedRide-2)",
                                  "Travel Cost per Distance, units:$/mi (SharedRide-3+)"]

mnl_specification["cars_per_licensed_drivers"] = [[1, 2, 3]]
mnl_names["cars_per_licensed_drivers"] = ["Autos per licensed drivers (All Auto Modes)"]

mnl_specification["total_travel_distance"] = [7, 8]
mnl_names["total_travel_distance"] = ['Travel Distance, units:mi (Walk)',
                                      'Travel Distance, units:mi (Bike)']

# mnl_specification["cross_bay"] = [[2, 3], [4, 5, 6]]
# mnl_names["cross_bay"] = ["Cross-Bay Tour (Shared Ride 2 & 3+)",
#                           "Cross-Bay Tour (All Transit Modes)"]
mnl_specification["cross_bay"] = [[2, 3]]
mnl_names["cross_bay"] = ["Cross-Bay Tour (Shared Ride 2 & 3+)"]

mnl_specification["household_size"] = [[2, 3]]
mnl_names["household_size"] = ['Household Size (Shared Ride 2 & 3+)']

mnl_specification["num_kids"] = [[2, 3]]
mnl_names["num_kids"] = ["Number of Kids in Household (Shared Ride 2 & 3+)"]

In [ ]:
# Estimate the basic MNL model, using the hessian and newton-conjugate gradient
mnl_model = pl.create_choice_model(data=bike_data_long,
                                   alt_id_col="mode_id",
                                   obs_id_col="observation_id",
                                   choice_col="choice",
                                   specification=mnl_specification,
                                   model_type="MNL",
                                   names=mnl_names)

num_vars = len(reduce(lambda x, y: x + y, mnl_names.values()))

# Note newton-cg used to ensure convergence to a point where gradient
# is essentially zero for all dimensions.
mnl_model.fit_mle(np.zeros(num_vars),
                  method="BFGS")

# Look at the estimation results
mnl_model.get_statsmodels_summary()

## Example Implementation 

## Declaring variables

In [ ]:
# Declaring all the needed variables for each of our functions

# Observation id column
observation_id_col = 'observation_id'

# Alternative id column
alternative_id_col = 'mode_id'

# Individual specific variables list
individual_specific_variables = ['num_kids', 'household_size',
                                 'num_cars', 'num_licensed_drivers']

# Alternative specific variables list
alternative_specific_variables = ['total_travel_time',
                                  'total_travel_distance',
                                  'total_travel_cost']

# Trip specific variables list
trip_specific_variables = ['cross_bay']

# Alternative name dictionary
alternative_name_dict = {1: 'drive_alone',
                         2: 'shared_2',
                         3: 'shared_3p',
                         4: 'wtw',
                         5: 'dtw',
                         6: 'wtd',
                         7: 'walk',
                         8: 'bike'}

# Variable type Dictionary
variable_type = {'num_kids': 'categorical',
                 'household_size': 'categorical',
                 'num_cars': 'categorical',
                 'num_licensed_drivers': 'categorical',
                 'cross_bay': 'categorical',
                 'total_travel_time': 'continuous',
                 'total_travel_distance': 'continuous',
                 'total_travel_cost': 'continuous'}

# Distribution to be explored for continuous variables
distributions = ['normal', 'alpha', 'beta', 'gamma', 'expon', 'gumbel']

## Implementation of Function

In [ ]:
# Generate a parameters dictionary from long
# format data
bike_data_params = FindLongDataDist(data_long=bike_data_long,
                                    alt_id_col=alternative_id_col,
                                    obs_id_col=observation_id_col,
                                    alt_spec=alternative_specific_variables,
                                    alt_name_dic=alternative_name_dict,
                                    ind_spec=individual_specific_variables,
                                    trip_spec=trip_specific_variables,
                                    var_types=variable_type,
                                    cont_dists=distributions)

In [ ]:
# Simulate data from the params_dict
wide_sim_data = SimDf(params_dict=bike_data_params, size=2000)

In [ ]:
# Explore the simulated data
wide_sim_data.head()

In [ ]:
# Simulate availability, add fake choice column
# and return final simulated data with availability
# and choices

wide_sim_data_availability = SimulateAvailability(bike_data_long,
                                     sim_data=wide_sim_data,
                                     obs_id_col=observation_id_col,
                                     alt_name_dict=alternative_name_dict)

# Simulate Choices 

## Convert Simulated Data from Wide to Long 

In [ ]:
ind_variables = ['num_kids', 'household_size',
                 'num_cars', 'num_licensed_drivers', 'cross_bay']


# Dictionary of Alternative Specific Variables
# TODO: verify whether all variables are needed
# for each alternative
alt_varying_variables = {u'total_travel_time': dict([(1, 'total_travel_time_drive_alone'),
                                                     (2, 'total_travel_time_shared_2'),
                                                     (3, 'total_travel_time_shared_3p'),
                                                     (4, 'total_travel_time_wtw'),
                                                     (5, 'total_travel_time_dtw'),
                                                     (6, 'total_travel_time_wtd'),
                                                     (7, 'total_travel_time_walk'),
                                                     (8, 'total_travel_time_bike')]),
                         u'total_travel_cost': dict([(1, 'total_travel_cost_drive_alone'),
                                                     (2, 'total_travel_cost_shared_2'),
                                                     (3, 'total_travel_cost_shared_3p'),
                                                     (4, 'total_travel_cost_wtw'),
                                                     (5, 'total_travel_cost_dtw'),
                                                     (6, 'total_travel_cost_wtd'),
                                                     (7, 'total_travel_cost_walk'),
                                                     (8, 'total_travel_cost_bike')]),
                         u'total_travel_distance': dict([(1, 'total_travel_distance_drive_alone'),
                                                         (2, 'total_travel_distance_shared_2'),
                                                         (3, 'total_travel_distance_shared_3p'),
                                                         (4, 'total_travel_distance_wtw'),
                                                         (5, 'total_travel_distance_dtw'),
                                                         (6, 'total_travel_distance_wtd'),
                                                         (7, 'total_travel_distance_walk'),
                                                         (8, 'total_travel_distance_bike')]),
                            }


# Dictionary of alternative availability variables
availability_variables = {1: 'drive_alone_AV',
                          2: 'shared_2_AV',
                          3: 'shared_3p_AV',
                          4: 'wtw_AV',
                          5: 'dtw_AV',
                          6: 'wtd_AV',
                          7: 'walk_AV',
                          8: 'bike_AV'}

##########
# Determine the columns for: alternative ids, the observation ids and the choice
##########
# The 'custom_alt_id' is the name of a column to be created in the long-format data
# It will identify the alternative associated with each row.
custom_alt_id = "mode_id"

# Create a custom id column that ignores the fact that this is a
# panel/repeated-observations dataset. Note the +1 ensures the id's start at one.
obs_id_column = "observation_id"

wide_sim_data_availability[obs_id_column] = np.arange(wide_sim_data.shape[0],
                                         dtype=int) + 1


# Create an empty choice column
choice_column = "sim_choice"

## Convert to Long Format Data

In [ ]:
# Convert data from wide to long
long_sim_data = pl.convert_wide_to_long(wide_sim_data_availability,
                                        ind_variables,
                                        alt_varying_variables,
                                        availability_variables,
                                        obs_id_column,
                                        choice_column,
                                        new_alt_id_name=custom_alt_id)

In [ ]:
# Create a cars per licensed drivers column
long_sim_data["cars_per_licensed_drivers"] = 0
long_sim_data.loc[long_sim_data.num_licensed_drivers > 0,
                  "cars_per_licensed_drivers"] = long_sim_data.num_cars / long_sim_data.num_licensed_drivers.astype(float)

In [ ]:
# Add a variable representing cost divided by distance
long_sim_data["cost_per_distance"] = 0
long_sim_data.loc[long_sim_data.mode_id.isin([1, 2, 3]),
                  "cost_per_distance"] = (long_sim_data.loc[long_sim_data.mode_id.isin([1, 2, 3]),
                                                            "total_travel_cost"] /
                                          long_sim_data.loc[long_sim_data.mode_id.isin([1, 2, 3]),
                                                            "total_travel_distance"])

## Simulate choices

In [ ]:
# Calculate probabilities for each alternative
# based on the estimated model
posterior_probs = mnl_model.predict(long_sim_data)

# Simulate choice data
long_sim_data['sim_choice'] = viz.simulate_choice_vector(posterior_probs,
                               long_sim_data['observation_id'].values)

## Estimate the model based on simulate choice data 

In [ ]:
# Estimate the basic MNL model, using the hessian and newton-conjugate gradient
mnl_model_sim = pl.create_choice_model(data=long_sim_data_choice,
                                       alt_id_col="mode_id",
                                       obs_id_col="observation_id",
                                       choice_col="sim_choice",
                                       specification=mnl_specification,
                                       model_type="MNL",
                                       names=mnl_names)

num_vars = len(reduce(lambda x, y: x + y, mnl_names.values()))
# Note newton-cg used to ensure convergence to a point where gradient
# is essentially zero for all dimensions.
mnl_model_sim.fit_mle(np.zeros(num_vars),
                      method="BFGS")

# Look at the estimation results
mnl_model_sim.get_statsmodels_summary()

# Repeat simulation many times 

### Set up all needed variables

In [ ]:
observation_id_col = 'observation_id'

alternative_id_col = 'mode_id'

variable_type = {'num_kids': 'categorical',
                 'household_size': 'categorical',
                 'num_cars': 'categorical',
                 'num_licensed_drivers': 'categorical'}

individual_specific_variables = ['num_kids', 'household_size',
                                 'num_cars', 'num_licensed_drivers']

alternative_specific_variables = ['total_travel_time', 'total_travel_distance', 'total_travel_cost']

trip_specific_variables = ['cross_bay']

alternative_name_dict = {1: 'drive_alone',
                         2: 'shared_2',
                         3: 'shared_3p',
                         4: 'wtw',
                         5: 'dtw',
                         6: 'wtd',
                         7: 'walk',
                         8: 'bike'}

variable_type = {'num_kids': 'categorical',
                 'household_size': 'categorical',
                 'num_cars': 'categorical',
                 'num_licensed_drivers': 'categorical',
                 'cross_bay': 'categorical',
                 'total_travel_time': 'continuous',
                 'total_travel_distance': 'continuous',
                 'total_travel_cost': 'continuous'}

distributions = ['normal', 'alpha', 'beta', 'gamma', 'expon', 'gumbel']

choice_column = "sim_choice"

custom_alt_id = "mode_id"

alt_varying_variables = {u'total_travel_time': dict([(1, 'total_travel_time_drive_alone'),
                                                     (2, 'total_travel_time_shared_2'),
                                                     (3, 'total_travel_time_shared_3p'),
                                                     (4, 'total_travel_time_wtw'),
                                                     (5, 'total_travel_time_dtw'),
                                                     (6, 'total_travel_time_wtd'),
                                                     (7, 'total_travel_time_walk'),
                                                     (8, 'total_travel_time_bike')]),
                         u'total_travel_cost': dict([(1, 'total_travel_cost_drive_alone'),
                                                     (2, 'total_travel_cost_shared_2'),
                                                     (3, 'total_travel_cost_shared_3p'),
                                                     (4, 'total_travel_cost_wtw'),
                                                     (5, 'total_travel_cost_dtw'),
                                                     (6, 'total_travel_cost_wtd'),
                                                     (7, 'total_travel_cost_walk'),
                                                     (8, 'total_travel_cost_bike')]),
                         u'total_travel_distance': dict([(1, 'total_travel_distance_drive_alone'),
                                                         (2, 'total_travel_distance_shared_2'),
                                                         (3, 'total_travel_distance_shared_3p'),
                                                         (4, 'total_travel_distance_wtw'),
                                                         (5, 'total_travel_distance_dtw'),
                                                         (6, 'total_travel_distance_wtd'),
                                                         (7, 'total_travel_distance_walk'),
                                                         (8, 'total_travel_distance_bike')]),
                        }


availability_variables = {1: 'drive_alone_AV',
                          2: 'shared_2_AV',
                          3: 'shared_3p_AV',
                          4: 'wtw_AV',
                          5: 'dtw_AV',
                          6: 'wtd_AV',
                          7: 'walk_AV',
                          8: 'bike_AV'}

bike_data_params = FindLongDataDist(data_long=bike_data_long,
                                    alt_id_col=alternative_id_col,
                                    obs_id_col=observation_id_col,
                                    alt_spec=alternative_specific_variables,
                                    alt_name_dic=alternative_name_dict,
                                    ind_spec=individual_specific_variables,
                                    trip_spec=trip_specific_variables,
                                    var_types=variable_type,
                                    cont_dists=distributions)

In [ ]:
simulation_size = np.random.randint(low=2000, high=3000, size=100)
sim_number = np.arange(1,101)
models_dictionary = defaultdict(dict)

for size, number in zip(simulation_size, sim_number):
    print('Simulation number', number , 'is in process...')
    print('------------------------------------------')

    # Simulate data from the params_dict
    wide_sim_data = SimDf(params_dict=bike_data_params,
                     size = size)

    # Simulate availability based on availability 
    # from long data and addd a fake choice column
    wide_sim_data_availability = SimulateAvailability(data_long=bike_data_long, 
                                         sim_data=sim_data, 
                                         obs_id_col=observation_id_col, 
                                         alt_name_dict=alternative_name_dict)

    # Add an observation_id column
    wide_sim_data[obs_id_column] = np.arange(wide_sim_data.shape[0],
                                            dtype=int) + 1
    
    # Convert simulated data from wide to long
    long_sim_data = pl.convert_wide_to_long(wide_data=wide_sim_data,
                                            ind_vars=ind_variables, 
                                            alt_specific_vars=alt_varying_variables, 
                                            availability_vars=availability_variables,
                                            obs_id_col=observation_id_col,
                                            choice_col=choice_column,
                                            new_alt_id_name=custom_alt_id)
    
    # Create a cars per licensed drivers column
    long_sim_data["cars_per_licensed_drivers"] = 0
    long_sim_data.loc[long_sim_data.num_licensed_drivers > 0,
                      "cars_per_licensed_drivers"] = long_sim_data.num_cars / long_sim_data.num_licensed_drivers.astype(float)

    # Add a variable representing cost divided by distance
    long_sim_data["cost_per_distance"] = 0
    long_sim_data.loc[long_sim_data.mode_id.isin([1, 2, 3]),
                      "cost_per_distance"] = (long_sim_data.loc[long_sim_data.mode_id.isin([1, 2, 3]),
                                                                "total_travel_cost"] /
                                              long_sim_data.loc[long_sim_data.mode_id.isin([1, 2, 3]),
                                                        "total_travel_distance"])
    
    # Calculate posterior probabilities for each alternative
    posterior_probs = mnl_model.predict(long_sim_data)

    # Simulate choices based on calculated probabilities
    long_sim_data['sim_choice'] = simulate_choice_vector(posterior_probs,
                                                         long_sim_data['observation_id'].values)

    # Estimate the basic MNL model, using the hessian and newton-conjugate gradient
    mnl_model_sim = pl.create_choice_model(data=long_sim_data,
                                           alt_id_col=alternative_id_col,
                                           obs_id_col=observation_id_col,
                                           choice_col=choice_column,
                                           specification=mnl_specification,
                                           model_type="MNL",
                                           names=mnl_names)

    num_vars = len(reduce(lambda x, y: x + y, mnl_names.values()))
    
    # Note newton-cg used to ensure convergence to a point where gradient 
    # is essentially zero for all dimensions. 
    mnl_model_sim.fit_mle(np.zeros(num_vars),
                          method="BFGS")

    # Store model
    models_dictionary[number] = mnl_model_sim
    
    print('Simulation number', number , 'is complete!')
    print('==========================================')
    print('==========================================')